## In the rough
In my previous notebook I tried to illustrate the use of recursion. With lots of help from the kaggle community via comments, we managed to improve my proposed solution, eventually matching the top score for the task. The final solution used no recursion!  

When I first wrote the code for `task096.py` I used regex and managed to get a solution of 367 bytes, surpassing the then top score of 373. Now the top score has moved up to 325. I am hoping that similar help from readers will get us to 325. Maybe without any regex.  

After years of trying to teach students to write elegant code, I find regex a bit of a **mess** in the context of a python solution to one of these tasks. Something in me fights against using regex. So this time I would actually be very happy with a regex-free improvement to a regex notebook.

In [ ]:
import sys
sys.path.append("/kaggle/input/google-code-golf-2025/code_golf_utils")
from code_golf_utils import *

## Rack and stack
First we take a look at the data for `task 96`.

In [ ]:
task_num = 96
show_examples(load_examples(task_num)['test']+load_examples(task_num)['arc-gen'][:2])

## Visualise First
I have annotated the output from the `train[0]` example. It is worthstudying the picture below before reading too much code. Also study the pandas dataframe below and make sure you are clear on how the numbers correspond to the output grid.  

In this example, **blue is the background** colour. Six concentric squares are placed on top of this background. The smallest square of one block is missing in this example, so the background blue shows through.

In [ ]:
from IPython.display import display, Image
display(Image(filename='/kaggle/input/task096-pics/task096.png', width=800))

In [ ]:
import pandas as pd
data =[
    [0,'Blue',1,1,0],
    [1,'Turquoise',8,3,0],
    [2,'Green',3,5,1],
    [3,'Purple',6,7,3],
    [4,'Red',2,9,3],
    [5,'Yellow',4,11,1],
      ]
df = pd.DataFrame(data, columns=['Square','Colour','Col Code' ,'Side', 'Middle Gap'])
df

## The all important dict
In this task finding this data is the tricky part, for which we will use regex. More on the regex later. First lets see where we are headed with the dict. Notice, in creating the dict, that I have created the gaps on the side of each square as `(gap + 1)//2`.  
  
This brings the possible gaps (all odd except the zero gap) of `0,1,3,5,7,...` down to `0,1,2,3,4,...` These sequential numbers correspond exactly to the minimum offset from the centre shown in the diagram above.

In [ ]:
S = {d[0]:((d[4]+1)//2,d[2]) for d in data}
S

## Chip it in from 30 metres
Once we have a dictionary of concentric squares `S` and know that the backgroud colour is `b=1` then we can colloct some input data and construct the output.

In [ ]:
R=range # Some golfer's housekeeping
b=1 # Background colour blue.
x=2*max(S)+1 # The size of the square output grid
[[b if(z[0]<(p:=S[z[1]])[0])else p[1]for j in R(x)if(z:=sorted([abs(j-x//2),abs(i-x//2)]))]for i in R(x)]

## 🎊

## Hacking out the rough
🤣 Is it sad that I look at that grid above and see colours everywhere?  
  
Before we can produce output like this, we need to use a **regex club** to hack out of the rough.  

Let's get some colour involved.

In [ ]:
example=load_examples(task_num)["test"][0] # Grab the input for our example
show_examples([example])
g = example["input"]

In [ ]:
import re
t=re.sub(r',\s','',str(g+[*zip(*g)]))# Prep the input for easier pattern search
t+=t[::-1] # This creates 4 versions of the input, seeing it from top, left, bottom and right
t 

## Background
We need the background colour colour so we can exclude it when searching for squares.

In [ ]:
b= int(max(t,key=t.count)) # Background colour as integer
b

In [ ]:
c=4 # This is the colour. We will find yellow patterns.
re.findall(f'{c}+',t)

## Halfway out
From the max length values in that regex search we know that the yellow square has corners with 5 yellow pixels.  

Now we look for this pattern:
`[yellow] [yellow] ... 1 [not yellow] or more ... [yellow]`  

It is important to start with 2 yellows so that we do not pick up the middle section of a square, which would be:
`[yellow] ... 1 [not yellow] or more ... [yellow]`   

The fact of this search being non-symmetrical, starting with 2 yellows and ending with one, is the reason we needed 4 copies of our string representation of the grid. In our current example (something of an edge case), we will only find the pattern `yy..x..y` when reading bottom to top.  

So let's code that pattern in regex:

In [ ]:
f=re.findall(f'{c}{c}[^]){c}]+{c}',t) 
print(f,len(f[0])-3) # We subtract the 3 yellows from our search result to get the length of the gap in the yellw square.

In [ ]:
show_examples([example])

## Corner ang Gap
So in this example the pattern is `[yellow][yellow][blue][yellow]' and we have a gap size of 1.  

We now have a **corner length of 4** and gap **length** of 1 for **yellow**. Now we loop through all the colours and construct a dictionary.

In [ ]:
# Initialise the dict with square 0 in background colour.
# This because not all examples have a tiny square at the centre.
B={0:(0,b)}
# Loop through all colours except the background
for c in{*R(10)}-{b}:
    if(v:=re.findall(f'{c}+',t)): # The corner size
        m=len((re.findall(f'{c}{c}[^]){c}]+{c}',t)+[3*'0'])[0])-3 # The gap size
        l=len(max(v,key=len))*(1,2)[m>0] # Squares [0,1] do not have a gap(m=0) so no need to double corner length.
        S[(l+m)//2]=((m+1)//2,c) # Compileng the dict
x=2*max(S)+1 # Output grid main dimension
x, S

## Putting
So we have constructed our original dict from the input grid g.  

Now we are ready to put it all together, or should I just say ready to put?

In [ ]:
%%writefile task.py
import re
def p(g,R=range,f=re.findall):
 t=re.sub(r',\s','',str(g+[*zip(*g)]));t+=t[::-1];b=int(max(t,key=t.count));B={0:(0,b)}
 for c in{*R(10)}-{b}:
  if(v:=f(f'{c}+',t)):
   m=len((f(f'{c}{c}[^]){c}]+{c}',t)+[3*'0'])[0])-3
   l=len(max(v,key=len))*(1,2)[m>0]
   B[(l+m)//2]=((m+1)//2,c)
 x=2*max(B)+1
 return[[b if(z[0]<(p:=B[z[1]])[0])else p[1]for j in R(x)if(z:=sorted([abs(j-x//2),abs(i-x//2)]))]for i in R(x)]

In [ ]:
verify_program(task_num, load_examples(task_num))

## Done 🎊
There we have it.  

zlib will compress the 418 bytes down to 363.